### **Lab7.ipynb**

#### Activity
- Data science - Laboratorio 7

#### Authors: 
- Diego Lemus
- Fabián Juárez


#### Date:
- 22 de Septiembre 2024


#### Repository:
- Link: https://github.com/FabianJuarez182/DS-LAB7.git

### Task 2 y 3

In [24]:
import json
import pandas as pd
import re
import string
from collections import Counter
import nltk
from nltk.corpus import stopwords

# Descargar las stopwords en español de nltk
nltk.download('stopwords')
spanish_stopwords = set(stopwords.words('spanish'))

# Cargar el archivo .txt con manejo de errores para JSON
def load_json_txt(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-16') as file:  # Cambiamos a utf-16
        for line in file:
            line = line.strip()  # Eliminar espacios en blanco y saltos de línea
            if line:  # Verificar que la línea no esté vacía
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError:
                    continue  # Ignorar líneas mal formadas
    return pd.DataFrame(data)

# Función para limpiar el texto
def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Eliminar caracteres especiales, menciones y números
    text = re.sub(r'\@\w+|\#|\d+', '', text)
    # Eliminar signos de puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Eliminar stopwords en español
    text = ' '.join([word for word in text.split() if word not in spanish_stopwords])
    return text

# Cargar el archivo de texto
df = load_json_txt('tioberny.txt')

# Aplicar la función de limpieza a la columna de texto
df['cleaned_text'] = df['rawContent'].apply(clean_text)

# Visualizar los datos limpiados
print(df[['rawContent', 'cleaned_text']].head())

# Calcular la frecuencia de las palabras
word_freq = Counter(" ".join(df['cleaned_text']).split())

# Mostrar las palabras más frecuentes después de la limpieza
print(word_freq.most_common(10))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\domot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                          rawContent  \
0  _\nConfirmado Compañeres,\n\nEl impuesto por l...   
1  #URGENTE Lo que los medios #faferos no informa...   
2  @IvanDuque @BArevalodeLeon Con que usaste PEGA...   
3  @IvanDuque @BArevalodeLeon Entre Ellos se enti...   
4  El presidente @BArevalodeLeon y la vicepreside...   

                                        cleaned_text  
0  confirmado compañeres impuesto —solo cuenta pa...  
1  urgente medios faferos informaron ayer acerca ...  
2  usaste pegasus espiar detractores obra narcisi...  
3         entienden bien cuadrando productivareunión  
4  presidente vicepresidenta participan sesión so...  
[('si', 462), ('presidente', 430), ('gobierno', 334), ('solo', 287), ('guatemala', 245), ('vos', 197), ('ser', 185), ('q', 181), ('corruptos', 177), ('bien', 171)]


In [27]:
import networkx as nx

# Función ajustada para extraer respuestas, menciones y retweets
def extract_interactions(row):
    interactions = []
    
    # Validar que 'user' y 'username' existan y sean cadenas
    if 'user' in row and isinstance(row['user'], dict) and 'username' in row['user']:
        user = row['user']['username'].lower() if isinstance(row['user']['username'], str) else None
    else:
        user = None

    # Extraer respuestas (si existen y son válidas)
    if 'inReplyToUser' in row and isinstance(row['inReplyToUser'], dict):
        replied_to_user = row['inReplyToUser'].get('username', None)
        if user and replied_to_user:
            interactions.append(('reply', user, replied_to_user.lower()))
    
    # Extraer menciones (si existen y son válidas)
    if 'mentionedUsers' in row and isinstance(row['mentionedUsers'], list):
        for mention in row['mentionedUsers']:
            if 'username' in mention and isinstance(mention['username'], str):
                interactions.append(('mention', user, mention['username'].lower()))
    
    # Extraer retweets (si existen y son válidos)
    if 'retweetedTweet' in row and isinstance(row['retweetedTweet'], dict):
        retweeted_user = row['retweetedTweet'].get('user', {}).get('username')
        if user and retweeted_user:
            interactions.append(('retweet', user, retweeted_user.lower()))
    
    return interactions

# Cargar los datos
df = load_json_txt('traficogt.txt')

# Aplicar la extracción de interacciones a cada tweet
df['interactions'] = df.apply(extract_interactions, axis=1)

# Descomponer las interacciones en un DataFrame
interactions = []
for interaction_list in df['interactions']:
    interactions.extend(interaction_list)

# Crear un DataFrame con las interacciones
interaction_df = pd.DataFrame(interactions, columns=['type', 'source', 'target'])

# Eliminar duplicados
interaction_df = interaction_df.drop_duplicates()

# Contar el número de menciones, respuestas y retweets
interaction_counts = interaction_df['type'].value_counts()

# Mostrar cuántas interacciones de cada tipo existen
print("Conteo de interacciones por tipo:")
print(interaction_counts)

# Función para crear y exportar un grafo para cada tipo de interacción
def create_and_export_graph(interaction_type, interaction_df):
    # Filtrar las interacciones por el tipo deseado (mención, respuesta o retweet)
    filtered_df = interaction_df[interaction_df['type'] == interaction_type]
    
    # Verificar si hay interacciones para este tipo
    if filtered_df.empty:
        print(f"No se encontraron interacciones de tipo {interaction_type}")
        return
    
    # Crear un grafo dirigido
    G = nx.DiGraph()
    
    # Añadir las interacciones filtradas al grafo
    for index, row in filtered_df.iterrows():
        G.add_edge(row['source'], row['target'], interaction_type=row['type'])
    
    # Exportar el grafo a Gephi en formato GEXF
    file_name = f"user_interactions_graph_{interaction_type}.gexf"
    nx.write_gexf(G, file_name)
    print(f"Grafo de {interaction_type} exportado a Gephi como '{file_name}'")

# Crear y exportar grafos para menciones, respuestas y retweets
create_and_export_graph('mention', interaction_df)
create_and_export_graph('reply', interaction_df)
create_and_export_graph('retweet', interaction_df)

Conteo de interacciones por tipo:
type
mention    7336
reply      3330
Name: count, dtype: int64
Grafo de mention exportado a Gephi como 'user_interactions_graph_mention.gexf'
Grafo de reply exportado a Gephi como 'user_interactions_graph_reply.gexf'
No se encontraron interacciones de tipo retweet


Se evidencia que no existen los retweet de manera explicita por lo que se utilizará la cnatidad de retweets de cada tweet para visualizar que tanto auge tienen y que tan influyente es el usuario.